# Projet 5A ENR

## Apprendre la stratégie journalière de stockage
### Mise en place d'un réseau de neurone

### Objectifs : 
- Estimer le coût d’un mix énergétique de manière rapide en limitant le temps de calcul de l'optimisation du stockage qui se fait aujourd’hui heure par heure. 

L’entrée du réseau peut être par exemple :

1) L’état des stocks des différents moyens de stockage (en valeur absolue ou en pourcentage) à 0h

2) La production nette de la journée (production totale de la journée - consommation totale)

3) le jour de l’année (entre 1 et 365)

Et les sorties seraient : l’état des stocks en fin journée des différents moyens de stockages et le cout de stockage de la journée.
Ceci n’st qu’une proposition. On peut aussi mettre en entrée, la variance de la production journalière qui peut avoir un rôle.

In [2]:
import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Prétraitement des données

- il faut genérer les données avec les modèles 

- Quel mix choisir ? Le choisir et l'importer ici =) 

- Le cout de stockage n'est t'il pas aussi dépendant du profil de consommation ? (estimation ADEME, Negawhatt, RTE ?) réponse : le profil de consommation intervient dans la production de la journée nette (qui n'est pas un partie logique)

- Comment on fait pour connaître la production de la journée idem pour la consommation ? Ducoup c'est online ? Quelle idée derrière la variance de la production journalière (+ de variance plus de stockage ?) ? 
- C'est pas un peu con de connaître exactement la production de la journée ? parceque ducoup le stock prédis ca va être sensiblement stock d'entree + prod de la journée : un peu étrange selon Flavie


- produire les données d'apprentissage contenant

    - 1 jour de l'année entre 1 et 365
    - état des stocks à h0
    - production de la journée
    - variance production journalière.
    - variance consomation journalière utile ? (peut être bien que oui aussi)





In [2]:
simu_opti_ademe = pd.read_csv("Project_Renewables_2050_Stage_Amandine/simulations/optimal_simulations/simu_opti_ademe/simu_opti_ademe_hourly_generation.csv")

In [3]:
simu_opti_ademe.drop(["Electricity demand","hour","reserve phs","reserve phs","reserve battery","reserve lake","Storage phs","Storage battery","Storage methanation"],inplace = True, axis =1)
#simu_opti_ademe.drop("hour",inplace = True, axis =1)

In [4]:
simu_opti_ademe

,offshore,onshore,pv,river,lake,biogas,phs,battery,methanation,Stored phs,Stored battery,Stored methanation
0,10.94,40.80,0.0,1.27,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8143.11
1,11.12,42.09,0.0,1.30,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8147.63
2,11.12,42.89,0.0,1.11,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8152.15
3,11.04,43.18,0.0,1.12,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8156.67
4,10.77,42.95,0.0,1.21,0.0,0.0,0.0,0.0,0.0,6.01,0.0,8161.19
...,...,...,...,...,...,...,...,...,...,...,...,...
8755,12.37,65.05,0.0,2.79,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8120.52
8756,12.47,65.34,0.0,2.72,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8125.04
8757,12.52,65.87,0.0,2.45,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8129.56
8758,12.53,65.79,0.0,2.24,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8134.08


In [5]:
#transforming Data from hour to day
#sum over generation
#hour 24 for stored

In [6]:
A =simu_opti_ademe[["offshore","onshore","pv","river","lake","biogas","phs","battery","methanation"]].groupby(np.arange(len(simu_opti_ademe))//24).sum()
A

,offshore,onshore,pv,river,lake,biogas,phs,battery,methanation
0,248.21,987.76,167.14,32.00,0.00,0.00,0.00,0.00,0.00
1,89.07,520.50,270.72,37.68,0.00,0.00,12.19,0.00,220.56
2,55.50,323.21,310.81,45.47,0.00,0.00,62.90,1.22,630.18
3,162.95,530.71,272.38,68.40,0.00,0.00,105.15,56.84,289.52
4,142.35,486.05,239.21,76.71,65.00,45.05,2.96,12.38,395.69
...,...,...,...,...,...,...,...,...,...
360,81.91,229.73,373.09,35.84,312.00,0.00,42.05,57.71,342.09
361,140.42,365.48,389.76,41.46,312.00,0.00,147.90,71.17,82.72
362,261.86,872.92,314.12,43.67,136.91,0.00,20.44,7.40,0.00
363,288.11,1405.12,235.34,46.40,0.00,0.00,0.00,0.00,0.00


In [7]:
B = simu_opti_ademe[["Stored phs","Stored battery","Stored methanation"]][::24]
B.reset_index(inplace = True)

In [8]:
B

,index,Stored phs,Stored battery,Stored methanation
0,0,0.00,0.00,8143.11
1,24,163.12,74.14,8251.58
2,48,180.00,74.14,7824.72
3,72,110.11,72.86,6424.34
4,96,1.96,13.04,5781.00
...,...,...,...,...
360,8640,136.08,41.32,8792.97
361,8664,133.30,17.25,8032.79
362,8688,22.71,7.79,7848.96
363,8712,0.00,0.00,7926.18


In [9]:
Inputs =[A,B]
Inputs =pd.concat(Inputs,axis =1)

In [10]:
Inputs

,offshore,onshore,pv,river,lake,biogas,phs,battery,methanation,index,Stored phs,Stored battery,Stored methanation
0,248.21,987.76,167.14,32.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,8143.11
1,89.07,520.50,270.72,37.68,0.00,0.00,12.19,0.00,220.56,24,163.12,74.14,8251.58
2,55.50,323.21,310.81,45.47,0.00,0.00,62.90,1.22,630.18,48,180.00,74.14,7824.72
3,162.95,530.71,272.38,68.40,0.00,0.00,105.15,56.84,289.52,72,110.11,72.86,6424.34
4,142.35,486.05,239.21,76.71,65.00,45.05,2.96,12.38,395.69,96,1.96,13.04,5781.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,81.91,229.73,373.09,35.84,312.00,0.00,42.05,57.71,342.09,8640,136.08,41.32,8792.97
361,140.42,365.48,389.76,41.46,312.00,0.00,147.90,71.17,82.72,8664,133.30,17.25,8032.79
362,261.86,872.92,314.12,43.67,136.91,0.00,20.44,7.40,0.00,8688,22.71,7.79,7848.96
363,288.11,1405.12,235.34,46.40,0.00,0.00,0.00,0.00,0.00,8712,0.00,0.00,7926.18


In [11]:
Inputs["days"]= Inputs.index.values +1
Inputs

,offshore,onshore,pv,river,lake,biogas,phs,battery,methanation,index,Stored phs,Stored battery,Stored methanation,days
0,248.21,987.76,167.14,32.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,8143.11,1
1,89.07,520.50,270.72,37.68,0.00,0.00,12.19,0.00,220.56,24,163.12,74.14,8251.58,2
2,55.50,323.21,310.81,45.47,0.00,0.00,62.90,1.22,630.18,48,180.00,74.14,7824.72,3
3,162.95,530.71,272.38,68.40,0.00,0.00,105.15,56.84,289.52,72,110.11,72.86,6424.34,4
4,142.35,486.05,239.21,76.71,65.00,45.05,2.96,12.38,395.69,96,1.96,13.04,5781.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,81.91,229.73,373.09,35.84,312.00,0.00,42.05,57.71,342.09,8640,136.08,41.32,8792.97,361
361,140.42,365.48,389.76,41.46,312.00,0.00,147.90,71.17,82.72,8664,133.30,17.25,8032.79,362
362,261.86,872.92,314.12,43.67,136.91,0.00,20.44,7.40,0.00,8688,22.71,7.79,7848.96,363
363,288.11,1405.12,235.34,46.40,0.00,0.00,0.00,0.00,0.00,8712,0.00,0.00,7926.18,364


In [12]:
# ajouter le coût du storage
# overnight cost $/kW
# CAPEX $/kWh
# Annuity $/kW/year > pas lié au stockage
# Fixed O&M $/MWh
# Storage annuity $/kWh/year

In [13]:
stor = ["phs","battery","methanation"]
tec = ["offshore","onshore","pv","river","lake","biogas","phs","battery","methanation"]

# Annualized power capex cost in M€/GW/year
capex = pd.read_csv("inputs/annuities.csv", index_col=0, squeeze=True, header=None)
# Existing capacities of the technologies by December 2017 in GW
capa_ex = pd.read_csv("inputs/existing_capas.csv", index_col=0, squeeze=True, header=None)
# Annualized energy capex cost of storage technologies in M€/GWh/year
capex_en = pd.read_csv("inputs/str_annuities.csv", index_col=0, squeeze=True, header=None)
# Annualized fixed operation and maintenance costs M€/GW/year
fOM = pd.read_csv("inputs/fO&M.csv", index_col=0, squeeze=True, header=None)
# Variable operation and maintenance costs in M€/GWh
vOM = pd.read_csv("inputs/vO&M.csv", index_col=0, squeeze=True, header=None)



# variables optimisés par les 5A 2021
s_capex = [0,0,84.16086]
s_capex = pd.Series(s_capex, index = stor)
# charging related fOM of storage in M€/GW/year
s_opex = [0,0,59.25]
s_opex = pd.Series(s_opex, index = stor)



/tmp/ipykernel_2863/950090184.py:5: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  capex = pd.read_csv("inputs/annuities.csv", index_col=0, squeeze=True, header=None)
/tmp/ipykernel_2863/950090184.py:7: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  capa_ex = pd.read_csv("inputs/existing_capas.csv", index_col=0, squeeze=True, header=None)
/tmp/ipykernel_2863/950090184.py:9: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  capex_en = pd.read_csv("inputs/str_annuities.csv", index_col=0, squeeze=True, header=None)
/tmp/ipykernel_2863/950090184.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the ca

In [14]:
vOM = vOM.rename(index={"Onshore": "onshore", "PV":"pv", "PHS":"phs", "Battery":"battery"})
fOM = fOM.rename(index={"Offshore": "offshore", "Onshore": "onshore", "PV":"pv", "PHS":"phs", "Battery":"battery"})
capex = capex.rename(index={"Offshore": "offshore", "Onshore": "onshore", "PV":"pv", "PHS":"phs", "Battery":"battery"})

In [15]:
tec = ["offshore","onshore","pv","river","lake","biogas","phs","battery","methanation"]
Q = pd.Series([12.64, 79.73, 121.98, 7.5, 13 ,32.93, 9.3, 20.08, 32.93], index = tec)
volume= pd.Series([180 ,74.14, 12499.09], index = stor) #volume du mix optimal 
S = pd.Series([9.3,20.08, 7.66], index = stor)
gene = Inputs[["phs","battery","methanation"]]

In [16]:
gene

,phs,battery,methanation
0,0.00,0.00,0.00
1,12.19,0.00,220.56
2,62.90,1.22,630.18
3,105.15,56.84,289.52
4,2.96,12.38,395.69
...,...,...,...
360,42.05,57.71,342.09
361,147.90,71.17,82.72
362,20.44,7.40,0.00
363,0.00,0.00,0.00


In [17]:
DAY_FIXED_COST = (sum((Q[tec] - capa_ex[tec]) * capex[tec] for tec in stor) \
+ sum(volume[storage_tecs] * capex_en[storage_tecs] for storage_tecs in stor)\
+ sum(Q[tec] * fOM[tec] for tec in stor)\
+ sum(S[storage_tecs] * (s_opex[storage_tecs] + s_capex[storage_tecs]) for storage_tecs in stor)\
)/(365*1000)

In [18]:
DAY_VAR_COST = sum((gene[tec] * vOM[tec]) for tec in stor)/1000

In [19]:
COST = DAY_VAR_COST +DAY_FIXED_COST

In [20]:
Inputs["Stored phs"] = Inputs["Stored phs"]/volume[0]*100
Inputs["Stored battery"] = Inputs["Stored battery"]/volume[1]*100
Inputs["Stored methanation"] = Inputs["Stored methanation"]/volume[2]*100

In [21]:
Inputs

,offshore,onshore,pv,river,lake,biogas,phs,battery,methanation,index,Stored phs,Stored battery,Stored methanation,days
0,248.21,987.76,167.14,32.00,0.00,0.00,0.00,0.00,0.00,0,0.000000,0.000000,65.149623,1
1,89.07,520.50,270.72,37.68,0.00,0.00,12.19,0.00,220.56,24,90.622222,100.000000,66.017446,2
2,55.50,323.21,310.81,45.47,0.00,0.00,62.90,1.22,630.18,48,100.000000,100.000000,62.602317,3
3,162.95,530.71,272.38,68.40,0.00,0.00,105.15,56.84,289.52,72,61.172222,98.273537,51.398462,4
4,142.35,486.05,239.21,76.71,65.00,45.05,2.96,12.38,395.69,96,1.088889,17.588346,46.251367,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,81.91,229.73,373.09,35.84,312.00,0.00,42.05,57.71,342.09,8640,75.600000,55.732398,70.348881,361
361,140.42,365.48,389.76,41.46,312.00,0.00,147.90,71.17,82.72,8664,74.055556,23.266793,64.266999,362
362,261.86,872.92,314.12,43.67,136.91,0.00,20.44,7.40,0.00,8688,12.616667,10.507149,62.796252,363
363,288.11,1405.12,235.34,46.40,0.00,0.00,0.00,0.00,0.00,8712,0.000000,0.000000,63.414057,364


In [22]:
#créer les Xtest, Ytest, Xtrain, Ytrain

In [23]:
Inputs["cost"]= COST*10000

In [24]:
X = Inputs[:-1]
X.drop(["cost"],inplace = True, axis =1)

/tmp/ipykernel_2863/2007421412.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(["cost"],inplace = True, axis =1)


In [25]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
XSc= scaler.transform(X)

XSc = pd.DataFrame(XSc, index =X.index, columns = X.columns)

In [26]:
Y = Inputs[1:]
Y = Y[["Stored phs","Stored battery","Stored methanation","cost"]]
#Y["cost"]=COST[1:]
Y

,Stored phs,Stored battery,Stored methanation,cost
1,90.622222,100.000000,66.017446,79.050926
2,100.000000,100.000000,62.602317,101.376259
3,61.172222,98.273537,51.398462,83.998748
4,1.088889,17.588346,46.251367,88.844983
5,0.000000,0.000000,39.216135,97.562159
...,...,...,...,...
360,75.600000,55.732398,70.348881,86.878929
361,74.055556,23.266793,64.266999,73.041486
362,12.616667,10.507149,62.796252,67.199040
363,0.000000,0.000000,63.414057,67.043640


In [27]:
from sklearn.model_selection import train_test_split 
X_train,X_test,Y_train,Y_test=train_test_split(XSc,Y,test_size=0.25,random_state=11)

## Model de réseau de neurones

Il s'agit d'un truc au hasard d'internet à affiner 
d'internet : 

'Pour la couche Dropout, on a diminué de 30% le nombre des données d’entrée afin d’éviter le phénomène du overfitting. La graine prend une valeur de 2 pour avoir des résultats plus reproductibles.' from https://www.cours-gratuit.com/tutoriel-python/tutoriel-python-matriser-les-rseaux-de-neurones-avec-keras

In [28]:
model = keras.Sequential([
layers.Dense(64, activation = 'relu', input_shape = [X_train.shape[1]]),
layers.Dropout(0.3, seed = 2),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(4)])

2022-11-08 12:49:41.564080: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-08 12:49:41.566407: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-08 12:49:41.606802: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


### Liste des optimizer à choisir

class Adadelta: Optimizer that implements the Adadelta algorithm.

class Adagrad: Optimizer that implements the Adagrad algorithm.

class Adam: Optimizer that implements the Adam algorithm.

class Adamax: Optimizer that implements the Adamax algorithm.

class Ftrl: Optimizer that implements the FTRL algorithm.

class Nadam: Optimizer that implements the NAdam algorithm.

class Optimizer: Base class for Keras optimizers.

class RMSprop: Optimizer that implements the RMSprop algorithm.

class SGD: Gradient descent (with momentum) optimizer.

## Liste des losses à choisir

class BinaryCrossentropy: Computes the cross-entropy loss between true labels and predicted labels.

class BinaryFocalCrossentropy: Computes the focal cross-entropy loss between true labels and predictions.

class CategoricalCrossentropy: Computes the crossentropy loss between the labels and predictions.

class CategoricalHinge: Computes the categorical hinge loss between y_true and y_pred.

class CosineSimilarity: Computes the cosine similarity between labels and predictions.

class Hinge: Computes the hinge loss between y_true and y_pred.

class Huber: Computes the Huber loss between y_true and y_pred.

class KLDivergence: Computes Kullback-Leibler divergence loss between y_true and y_pred.

class LogCosh: Computes the logarithm of the hyperbolic cosine of the prediction error.

class Loss: Loss base class.

class MeanAbsoluteError: Computes the mean of absolute difference between labels and predictions.

class MeanAbsolutePercentageError: Computes the mean absolute percentage error between y_true and y_pred.

class MeanSquaredError: Computes the mean of squares of errors between labels and predictions.

class MeanSquaredLogarithmicError: Computes the mean squared logarithmic error between y_true and y_pred.

class Poisson: Computes the Poisson loss between y_true and y_pred.

class Reduction: Types of loss reduction.

class SparseCategoricalCrossentropy: Computes the crossentropy loss between the labels and predictions.

class SquaredHinge: Computes the squared hinge loss between y_true and y_pred.

## Choisir une metrics

In [29]:
optimiseur = tf.keras.optimizers.RMSprop(learning_rate = 0.001)
model.compile(loss = tf.keras.losses.MeanSquaredError(),
optimizer = optimiseur,
metrics = ['accuracy'])

training = model.fit (X_train, Y_train, epochs = 70, validation_split = 0.2)

2022-11-08 12:49:41.803443: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-11-08 12:49:41.839549: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2899885000 Hz


Epoch 1/70
7/7 [==============================] - 3s 235ms/step - loss: 2800.7840 - accuracy: 0.2984 - val_loss: 2412.8767 - val_accuracy: 0.3455
Epoch 2/70
7/7 [==============================] - 0s 14ms/step - loss: 2578.6255 - accuracy: 0.3763 - val_loss: 1546.9099 - val_accuracy: 0.3455
Epoch 3/70
7/7 [==============================] - 0s 20ms/step - loss: 1391.8722 - accuracy: 0.3639 - val_loss: 754.7174 - val_accuracy: 0.3455
Epoch 4/70
7/7 [==============================] - 0s 42ms/step - loss: 930.3117 - accuracy: 0.4927 - val_loss: 578.5130 - val_accuracy: 0.5818
Epoch 5/70
7/7 [==============================] - 0s 33ms/step - loss: 715.6800 - accuracy: 0.5327 - val_loss: 554.8852 - val_accuracy: 0.5818
Epoch 6/70
7/7 [==============================] - 0s 20ms/step - loss: 725.0987 - accuracy: 0.4841 - val_loss: 497.8064 - val_accuracy: 0.5818
Epoch 7/70
7/7 [==============================] - 0s 14ms/step - loss: 686.4309 - accuracy: 0.5227 - val_loss: 485.1674 - val_accuracy: 

Epoch 58/70
7/7 [==============================] - 0s 18ms/step - loss: 262.5144 - accuracy: 0.8137 - val_loss: 218.4514 - val_accuracy: 0.8000
Epoch 59/70
7/7 [==============================] - 0s 17ms/step - loss: 220.1209 - accuracy: 0.7874 - val_loss: 237.7552 - val_accuracy: 0.8000
Epoch 60/70
7/7 [==============================] - 0s 19ms/step - loss: 248.8239 - accuracy: 0.7891 - val_loss: 220.6730 - val_accuracy: 0.8000
Epoch 61/70
7/7 [==============================] - 0s 19ms/step - loss: 231.5949 - accuracy: 0.8007 - val_loss: 227.4564 - val_accuracy: 0.7818
Epoch 62/70
7/7 [==============================] - 0s 22ms/step - loss: 246.6319 - accuracy: 0.8046 - val_loss: 219.5604 - val_accuracy: 0.8182
Epoch 63/70
7/7 [==============================] - 0s 16ms/step - loss: 221.8139 - accuracy: 0.8290 - val_loss: 209.8303 - val_accuracy: 0.8000
Epoch 64/70
7/7 [==============================] - 0s 18ms/step - loss: 241.2417 - accuracy: 0.7918 - val_loss: 232.7655 - val_accuracy:

In [30]:
score =model.evaluate(X_test,Y_test)

3/3 [==============================] - 0s 14ms/step - loss: 219.8035 - accuracy: 0.8462


In [31]:
pd.DataFrame(model.predict(X_test))

,0,1,2,3
0,24.720945,10.460505,72.096848,73.114548
1,17.776756,7.839264,71.587379,72.259148
2,36.412880,15.278636,59.088215,70.676628
3,15.233307,7.902519,82.315681,72.262665
4,18.493984,7.651507,48.712612,70.830803
...,...,...,...,...
86,24.516983,13.837086,82.362389,68.164734
87,29.498497,12.362167,64.688385,73.484955
88,41.388847,20.210968,65.537689,71.302185
89,14.983474,6.627334,62.283195,69.366852


In [32]:
Y_test

,Stored phs,Stored battery,Stored methanation,cost
261,17.222222,0.000000,72.771778,68.52267
229,3.983333,0.000000,67.231294,68.20095
155,37.438889,24.035608,56.135767,67.39917
318,13.194444,0.000000,79.056875,67.04364
140,0.000000,0.000000,44.062168,67.04364
...,...,...,...,...
364,0.000000,0.000000,64.281880,67.04364
165,27.477778,4.504991,63.785924,68.58945
219,0.000000,0.000000,58.601626,68.16063
196,0.000000,0.000000,62.050917,67.04364


## Ajout des autres années de 2000 à 2019

In [8]:
simu_opti_ademe2000 = pd.read_csv("simulations/simu_opti_ademe2000/simu_opti_ademe2000_hourly_generation.csv")
simu_opti_ademe2001 = pd.read_csv("simulations/simu_opti_ademe2001/simu_opti_ademe2001_hourly_generation.csv")
simu_opti_ademe2002 = pd.read_csv("simulations/simu_opti_ademe2002/simu_opti_ademe2002_hourly_generation.csv")
simu_opti_ademe2003 = pd.read_csv("simulations/simu_opti_ademe2003/simu_opti_ademe2003_hourly_generation.csv")
simu_opti_ademe2004 = pd.read_csv("simulations/simu_opti_ademe2004/simu_opti_ademe2004_hourly_generation.csv")
simu_opti_ademe2005 = pd.read_csv("simulations/simu_opti_ademe2005/simu_opti_ademe2005_hourly_generation.csv")
simu_opti_ademe2006 = pd.read_csv("Project_Renewables_2050_Stage_Amandine/simulations/optimal_simulations/simu_opti_ademe/simu_opti_ademe_hourly_generation.csv")
simu_opti_ademe2007= pd.read_csv("simulations/simu_opti_ademe2007/simu_opti_ademe2007_hourly_generation.csv")
simu_opti_ademe2008 = pd.read_csv("simulations/simu_opti_ademe2008/simu_opti_ademe2008_hourly_generation.csv")
simu_opti_ademe2009 = pd.read_csv("simulations/simu_opti_ademe2009/simu_opti_ademe2009_hourly_generation.csv")
simu_opti_ademe2010 = pd.read_csv("simulations/simu_opti_ademe2010/simu_opti_ademe2010_hourly_generation.csv")
simu_opti_ademe2011 = pd.read_csv("simulations/simu_opti_ademe2011/simu_opti_ademe2011_hourly_generation.csv")
simu_opti_ademe2012 = pd.read_csv("simulations/simu_opti_ademe2012/simu_opti_ademe2012_hourly_generation.csv")
simu_opti_ademe2013 = pd.read_csv("simulations/simu_opti_ademe2013/simu_opti_ademe2013_hourly_generation.csv")
#simu_opti_ademe2014 = pd.read_csv("simulations/simu_opti_ademe2014/simu_opti_ademe2014_hourly_generation.csv") problème pour simuler 2014 :/
simu_opti_ademe2015 = pd.read_csv("simulations/simu_opti_ademe2015/simu_opti_ademe2015_hourly_generation.csv")
simu_opti_ademe2016 = pd.read_csv("simulations/simu_opti_ademe2016/simu_opti_ademe2016_hourly_generation.csv")
simu_opti_ademe2017 = pd.read_csv("simulations/simu_opti_ademe2017/simu_opti_ademe2017_hourly_generation.csv")
simu_opti_ademe2018 = pd.read_csv("simulations/simu_opti_ademe2018/simu_opti_ademe2018_hourly_generation.csv")
simu_opti_ademe2019 = pd.read_csv("simulations/simu_opti_ademe2019/simu_opti_ademe2019_hourly_generation.csv")

In [16]:
simu_opti_ademe2000

,hour,offshore,onshore,pv,river,lake,biogas,phs,battery,methanation,Storage phs,Storage battery,Storage methanation,Stored phs,Stored battery,Stored methanation,reserve phs,reserve battery,reserve lake,Electricity demand
0,0,5.82,15.78,0.0,1.27,0.0,0.0,0.0,0.0,5.82,-0.00,-0.0,-0.00,180.00,74.14,10059.72,7.44,0.00,0.00,28.69
1,1,5.74,16.61,0.0,1.30,0.0,0.0,0.0,0.0,7.56,-0.00,-0.0,-0.00,180.00,74.14,10046.78,0.00,7.47,0.00,31.21
2,2,5.61,17.10,0.0,1.11,0.0,0.0,0.0,0.0,3.88,-0.00,-0.0,-0.00,180.00,74.14,10029.97,0.00,0.00,7.43,27.70
3,3,5.64,17.78,0.0,1.12,0.0,0.0,0.0,0.0,3.45,-0.00,-0.0,-0.00,180.00,74.14,10021.35,0.00,7.44,0.00,27.99
4,4,5.72,18.42,0.0,1.21,0.0,0.0,0.0,0.0,2.81,-0.00,-0.0,-0.00,180.00,74.14,10013.68,0.00,7.44,0.00,28.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,8755,5.11,32.99,0.0,2.79,13.0,0.0,0.0,0.0,0.00,-9.30,-0.0,-5.98,137.40,74.14,10055.09,0.00,7.55,0.00,38.61
8756,8756,4.17,30.48,0.0,2.72,13.0,0.0,0.0,0.0,0.00,-9.30,-0.0,-1.74,146.24,74.14,10058.62,0.00,7.56,0.00,39.34
8757,8757,3.19,27.04,0.0,2.45,13.0,0.0,0.0,0.0,0.00,-8.98,-0.0,-0.00,155.07,74.14,10059.64,0.00,7.53,0.00,36.71
8758,8758,2.43,23.58,0.0,2.24,13.0,0.0,0.0,0.0,0.00,-7.97,-0.0,-0.00,163.60,74.14,10059.64,7.50,0.00,0.00,33.29


In [17]:
Simu_Opti_Ademe= pd.concat([simu_opti_ademe2000,simu_opti_ademe2001,simu_opti_ademe2002,simu_opti_ademe2003,
                            simu_opti_ademe2004,simu_opti_ademe2005,simu_opti_ademe2006,simu_opti_ademe2007,
                            simu_opti_ademe2008,simu_opti_ademe2009,simu_opti_ademe2010,simu_opti_ademe2011,
                           simu_opti_ademe2012,simu_opti_ademe2013,simu_opti_ademe2015,simu_opti_ademe2016,
                           simu_opti_ademe2017,simu_opti_ademe2018,simu_opti_ademe2019], axis =0, ignore_index =True)



In [18]:
Simu_Opti_Ademe

,hour,offshore,onshore,pv,river,lake,biogas,phs,battery,methanation,Storage phs,Storage battery,Storage methanation,Stored phs,Stored battery,Stored methanation,reserve phs,reserve battery,reserve lake,Electricity demand
0,0,5.82,15.78,0.0,1.27,0.0,0.0,0.0,0.0,5.82,-0.00,-0.0,-0.00,180.00,74.14,10059.72,7.44,0.00,0.00,28.69
1,1,5.74,16.61,0.0,1.30,0.0,0.0,0.0,0.0,7.56,-0.00,-0.0,-0.00,180.00,74.14,10046.78,0.00,7.47,0.00,31.21
2,2,5.61,17.10,0.0,1.11,0.0,0.0,0.0,0.0,3.88,-0.00,-0.0,-0.00,180.00,74.14,10029.97,0.00,0.00,7.43,27.70
3,3,5.64,17.78,0.0,1.12,0.0,0.0,0.0,0.0,3.45,-0.00,-0.0,-0.00,180.00,74.14,10021.35,0.00,7.44,0.00,27.99
4,4,5.72,18.42,0.0,1.21,0.0,0.0,0.0,0.0,2.81,-0.00,-0.0,-0.00,180.00,74.14,10013.68,0.00,7.44,0.00,28.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166435,8755,5.11,32.99,0.0,2.79,13.0,0.0,0.0,0.0,0.00,-9.30,-0.0,-5.98,137.40,74.14,10055.09,0.00,7.55,0.00,38.61
166436,8756,4.17,30.48,0.0,2.72,13.0,0.0,0.0,0.0,0.00,-9.30,-0.0,-1.74,146.24,74.14,10058.62,0.00,7.56,0.00,39.34
166437,8757,3.19,27.04,0.0,2.45,13.0,0.0,0.0,0.0,0.00,-8.98,-0.0,-0.00,155.07,74.14,10059.64,0.00,7.53,0.00,36.71
166438,8758,2.43,23.58,0.0,2.24,13.0,0.0,0.0,0.0,0.00,-7.97,-0.0,-0.00,163.60,74.14,10059.64,7.50,0.00,0.00,33.29
